In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt

# Baseline 

In [3]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps['Stats Model MAE'].data,nb.scraps['Catboost MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["Stats Model","Catboost"])
#baseLine_data = np.reshape(baseLine_data,(2,10))
display(df)
print(df.mean())

,Stats Model,Catboost
0,0.394439,0.156707
1,0.368530,0.130645
2,0.353290,0.073173
3,0.497611,0.146667
4,0.456710,0.141902
5,0.362967,0.089064
6,0.372887,0.325105
7,0.323525,0.079777
8,0.358416,0.146612
9,0.569174,0.225214


Stats Model    0.405755
Catboost       0.151487
dtype: float64


# GAN Model

In [4]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEAN:")
print(df.mean(axis = 0))
gan_data = np.array(gan_data)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,0.052149,0.152866,1.020008,3.057312
1,0.123697,0.229275,1.572754,4.585508
2,0.121998,0.224301,1.561952,4.486017
3,0.062488,0.192243,1.117595,3.844851
4,0.124140,0.252486,1.575309,5.049725
5,0.034784,0.124183,0.833122,2.483661
6,0.107623,0.224102,1.466874,4.482047
7,0.047333,0.157143,0.972630,3.142854
8,0.187382,0.269006,1.935102,5.380119
9,0.042565,0.149225,0.922446,2.984504


MEAN:
MSE                   0.090416
MAE                   0.197483
Euclidean Distance    1.297779
Manhattan Distance    3.949660
dtype: float64


# ABC_GAN Analysis

## ABC Pre-generator - Catboost 


In [5]:
book = sb.read_notebooks("./ABC_GAN_Catboost")
paramVal = [1,0.1,0.01]
abc_mae = [[] for i in range(3)]
abc_mae_skip = [[] for i in range(3)]
abc_mae_mean = [[] for i in range(3)]
abc_mae_skip_mean = [[] for i in range(3)]
abc_weights = [[] for i in range(3)]
prior_model = [[] for i in range(3)]
abc_pre_generator = [[] for i in range(3)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    
    #Divide data according to parameters 
    for i in range(3):
        if paramVar == paramVal[i]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [6]:
for i in range(3):
    data = []
    for j in range(len(abc_weights[i])):
        data.append([paramVal[i],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data, columns = ['Variance','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.mean(axis=0))
    print("-------------------------------------------------------------")

,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.36519,0.82244,0.85718,0.18133,0.24190
1,1,0.25950,0.96037,0.96353,0.24056,0.24025
2,1,0.27777,1.09340,0.12461,0.12237,0.14317
3,1,0.38670,1.02542,0.13763,0.23180,0.19890
4,1,0.38725,0.80040,0.43624,0.16629,0.14163
5,1,0.39898,0.91363,0.13002,0.18368,0.15765
6,1,0.35769,0.70796,0.87333,0.31402,0.24247
7,1,0.39044,1.05778,0.17070,0.24552,0.25973
8,1,0.24689,0.67001,0.16942,0.23309,0.21876
9,1,0.40308,1.18803,0.80421,0.13265,0.17569


Variance                         1.000000
Prior Model MAE                  0.347349
ABC pre-generator MAE            0.923944
Skip Node weight                 0.466687
ABC GAN MAE                      0.205131
ABC_GAN MAE (skip connection)    0.202014
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.33976,0.35797,0.11676,0.31875,0.27006
1,0.1,0.25730,0.28768,0.20480,0.21887,0.24233
2,0.1,0.38077,0.37640,0.12946,0.27418,0.29809
3,0.1,0.33442,0.34717,0.68735,0.25758,0.31091
4,0.1,0.50919,0.54836,0.59553,0.38490,0.29446
5,0.1,0.41282,0.45876,0.72519,0.35048,0.35649
6,0.1,0.42055,0.43933,0.21397,0.30466,0.26079
7,0.1,0.31617,0.33700,0.28144,0.27400,0.31562
8,0.1,0.31767,0.33774,0.45530,0.25659,0.26754
9,0.1,0.27939,0.29228,0.17107,0.15543,0.19037


Variance                         0.100000
Prior Model MAE                  0.356803
ABC pre-generator MAE            0.378268
Skip Node weight                 0.358087
ABC GAN MAE                      0.279542
ABC_GAN MAE (skip connection)    0.280665
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.41982,0.42283,0.02881,0.37347,0.38408
1,0.01,0.27806,0.27762,0.02157,0.26744,0.27102
2,0.01,0.25623,0.25844,0.00000,0.21213,0.25667
3,0.01,0.31094,0.31209,0.04464,0.30470,0.30846
4,0.01,0.33483,0.33391,0.45408,0.30495,0.31293
5,0.01,0.19979,0.20244,0.02696,0.20053,0.19677
6,0.01,0.25448,0.25198,0.24304,0.25751,0.24670
7,0.01,0.38354,0.38191,0.33916,0.37152,0.38785
8,0.01,0.32563,0.32444,0.00000,0.26165,0.32574
9,0.01,0.30432,0.30493,0.00000,0.26269,0.30470


Variance                         0.010000
Prior Model MAE                  0.306765
ABC pre-generator MAE            0.307060
Skip Node weight                 0.115826
ABC GAN MAE                      0.281659
ABC_GAN MAE (skip connection)    0.299491
dtype: float64
-------------------------------------------------------------


## ABC Pre-generator - Stats 


In [7]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [1,0.1,0.01]
abc_mae = [[] for i in range(3)]
abc_mae_skip = [[] for i in range(3)]
abc_mae_mean = [[] for i in range(3)]
abc_mae_skip_mean = [[] for i in range(3)]
abc_weights = [[] for i in range(3)]
prior_model = [[] for i in range(3)]
abc_pre_generator = [[] for i in range(3)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    
    #Divide data according to parameters 
    for i in range(3):
        if paramVar == paramVal[i]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [8]:
for i in range(3):
    data = []
    for j in range(len(abc_weights[i])):
        data.append([paramVal[i],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data, columns = ['Variance','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.mean(axis=0))
    print("-------------------------------------------------------------")

,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.50891,0.95473,0.99317,0.32740,0.21742
1,1,0.55453,0.98361,0.94769,0.28913,0.16422
2,1,0.41843,0.67262,0.99496,0.31799,0.21815
3,1,0.52749,0.82503,1.00000,0.27268,0.24384
4,1,0.37072,0.75668,0.92180,0.26260,0.15983
5,1,0.46808,0.97237,0.99569,0.26564,0.29963
6,1,0.26985,1.00872,0.96113,0.20930,0.19894
7,1,0.32584,0.80817,0.89217,0.26428,0.20468
8,1,0.38052,1.05728,0.97678,0.28408,0.30569
9,1,0.39524,0.91350,0.86179,0.37848,0.12205


Variance                         1.000000
Prior Model MAE                  0.421961
ABC pre-generator MAE            0.895271
Skip Node weight                 0.954519
ABC GAN MAE                      0.287158
ABC_GAN MAE (skip connection)    0.213444
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.43601,0.42392,0.23762,0.40460,5170.38506
1,0.1,0.45853,0.46924,0.26108,0.23921,0.18820
2,0.1,0.43518,0.48480,0.18357,0.29470,0.22892
3,0.1,0.37966,0.43818,0.17162,0.30856,0.14591
4,0.1,0.33140,0.30998,0.17233,0.25911,0.18143
5,0.1,0.36843,0.34918,0.14877,0.20677,0.19369
6,0.1,0.38729,0.38222,0.17439,0.30862,0.16367
7,0.1,0.36265,0.33042,0.25029,0.25403,0.12171
8,0.1,0.34082,0.33334,0.18623,0.24730,0.19920
9,0.1,0.38228,0.44806,0.20840,0.29567,0.16809


Variance                           0.100000
Prior Model MAE                    0.388225
ABC pre-generator MAE              0.396933
Skip Node weight                   0.199429
ABC GAN MAE                        0.281856
ABC_GAN MAE (skip connection)    517.197587
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.34018,0.33796,0.15675,0.22513,80.73940
1,0.01,0.58179,0.58131,0.17463,0.37683,0.33590
2,0.01,0.38415,0.38487,0.14793,0.28391,0.22742
3,0.01,0.39787,0.39692,0.10145,0.19094,0.19563
4,0.01,0.41789,0.41517,0.14969,0.30512,103.85069
5,0.01,0.34601,0.34534,0.16624,0.25881,0.26452
6,0.01,0.35317,0.35217,0.15335,0.30854,0.19807
7,0.01,0.33657,0.33382,0.10982,0.19203,0.16317
8,0.01,0.36806,0.36829,0.08112,0.31559,39.94542
9,0.01,0.38586,0.38620,0.00000,0.34596,0.38660


Variance                          0.010000
Prior Model MAE                   0.391156
ABC pre-generator MAE             0.390206
Skip Node weight                  0.124099
ABC GAN MAE                       0.280284
ABC_GAN MAE (skip connection)    22.630680
dtype: float64
-------------------------------------------------------------
